# Iterated Networks

Simple, deep networks with few parameters that perform well and train quickly.

## Recipe

 1. Create a layer that has the same outputs as inputs.
 2. Apply that layer over and over, feeding output back into input.
 3. Optionally add some skip connections with the first imput.

## Results
Comparable performance to a simple conv net, but trains faster. Also more iterations -> better results.

## TODO
 
 1. What is going on between the iterations?
 2. Can we train with just one or a few iterations and then deploy with many?
 3. Training method of unrolling loops looks superficially like unrolling through time in an RNN, is there anything to
    that?
 4. Compare with a more realistic competition, and on a more diverse data set.
 5. Fully connected networks?

In [1]:
from models import (
    train,
    iterated_model_unrolled, 
    iterated_model_while_op, 
    cnn_model
)
import tensorflow as tf
import numpy as np
from functools import partial
from utils import Timer

tf.logging.set_verbosity(tf.logging.ERROR)

%rm -r ./data/

In [2]:
mnist = tf.contrib.learn.datasets.load_dataset("mnist")
train_data = mnist.train.images  # Returns np.array
train_labels = np.asarray(mnist.train.labels, dtype=np.int32)
eval_data = mnist.test.images  # Returns np.array
eval_labels = np.asarray(mnist.test.labels, dtype=np.int32)

steps = 20e3

def train_on_mnist(*args, **kwargs):
    with Timer() as timer:
        results = train(eval_data, eval_labels, train_data, train_labels, steps, *args, **kwargs)
    return (timer.elapsed_seconds, *results)

Extracting MNIST-data/train-images-idx3-ubyte.gz
Extracting MNIST-data/train-labels-idx1-ubyte.gz
Extracting MNIST-data/t10k-images-idx3-ubyte.gz
Extracting MNIST-data/t10k-labels-idx1-ubyte.gz


In [3]:
cnn_model_results = train_on_mnist(cnn_model)
cnn_model_results

(650.150568436,
 {'accuracy': 0.9677, 'loss': 0.104079105, 'global_step': 20000})

In [4]:
iterated_model_unrolled_results = train_on_mnist(iterated_model_unrolled, params={'iterations': 5, 'channels': 5})
iterated_model_unrolled_results

(824.6978443209991,
 {'accuracy': 0.9833, 'loss': 0.056544002, 'global_step': 20000})

In [5]:
iterated_model_while_op_results = train_on_mnist(iterated_model_while_op, params={'iterations': 5, 'channels': 5})
iterated_model_while_op_results

(1440.6643616149995,
 {'accuracy': 0.985, 'loss': 0.050225332, 'global_step': 20000})

In [6]:
def iteration_study(iterations):
    return train_on_mnist(
        iterated_model_unrolled,
        f'iterated_model_unrolled_iterations_{iterations}',
        {
            'iterations': iterations,
            'channels': 5
        }
    )

In [7]:
iterations = [
    0,  # Trivial case
    1,
    2,
    5,
    10,
    20
]
iteration_study_results = list(map(iteration_study, iterations))
iteration_study_results

[(235.80641283800105,
  {'accuracy': 0.938, 'loss': 0.21855783, 'global_step': 20000}),
 (355.8324502329997,
  {'accuracy': 0.9721, 'loss': 0.09564532, 'global_step': 20000}),
 (472.53805637500227,
  {'accuracy': 0.9791, 'loss': 0.06557379, 'global_step': 20000}),
 (824.4319346169978,
  {'accuracy': 0.9815, 'loss': 0.056496806, 'global_step': 20000}),
 (1414.1717638779992,
  {'accuracy': 0.986, 'loss': 0.04564997, 'global_step': 20000}),
 (2591.5159817189997,
  {'accuracy': 0.9855, 'loss': 0.05003326, 'global_step': 20000})]

In [8]:
def channel_study(channels):
    return train_on_mnist(
        iterated_model_unrolled,
        f'iterated_model_unrolled_channels_{channels}',
        {
            'iterations': 5,
            'channels': channels
        }
    )

In [9]:
channels = [
    1,
    2,
    5,
    10,
    20
]
channel_study_results = list(map(channel_study, channels))
channel_study_results

[(403.0317638680026,
  {'accuracy': 0.9749, 'loss': 0.08136818, 'global_step': 20000}),
 (495.5032902210005,
  {'accuracy': 0.9798, 'loss': 0.067920916, 'global_step': 20000}),
 (824.7462080999976,
  {'accuracy': 0.9835, 'loss': 0.051396906, 'global_step': 20000}),
 (1259.4520230920025,
  {'accuracy': 0.9858, 'loss': 0.053202346, 'global_step': 20000}),
 (2182.091704798,
  {'accuracy': 0.985, 'loss': 0.05615745, 'global_step': 20000})]